In [42]:
'''
  code by Minho Ryu @bzantium
  
'''
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

char_arr = list('abcdefghijklmnopqrstuvwxyz')
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
vocab_size = len(word_dict)
n_embed = 20
n_class = len(word_dict)
n_step = 3
n_hidden = 128

def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data:
        input_batch.append([word_dict[n] for n in seq[:-1]])
        target_batch.append([word_dict[seq[-1]]])
        
    return input_batch, target_batch

# Model
class TextLSTM(object):
    def __init__(self, sess, vocab_size, n_embed, n_step, n_hidden, n_class):
        self.sess = sess
        self.vocab_size = vocab_size
        self.n_embed = n_embed
        self.n_step = n_step
        self.n_hidden = n_hidden
        self.n_class = n_class
        self._build_model()
    
    def _build_model(self):
        with tf.variable_scope("placeholder"):
            self.X = tf.placeholder(tf.int32, [None, self.n_step]) # [batch_size, n_step]
            self.Y = tf.placeholder(tf.int32, [None, 1])         # [batch_size, 1]
        
        with tf.variable_scope("embedding"):
            W = tf.Variable(tf.random_normal([self.vocab_size, self.n_embed]))
            b = tf.Variable(tf.random_normal([self.n_embed]))
            embedded = tf.nn.embedding_lookup(W, self.X)
        
        with tf.variable_scope("rnn"):
            cell = tf.nn.rnn_cell.BasicLSTMCell(self.n_hidden)
            outputs, states = tf.nn.dynamic_rnn(cell, embedded, dtype=tf.float32)
            outputs = tf.transpose(outputs, [1, 0, 2]) # [n_step, batch_size, n_hidden]
            outputs = outputs[-1] # [batch_size, n_hidden]
        
        with tf.variable_scope("output"):
            W = tf.Variable(tf.random_normal([self.n_hidden, self.n_class]))
            b = tf.Variable(tf.random_normal([self.n_class]))
            logits = tf.matmul(outputs, W) + b
        
        logits = tf.expand_dims(logits, 1)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(0.001).minimize(self.cost)
        self.prediction = tf.cast(tf.argmax(logits, -1), tf.int32)

        tf.global_variables_initializer().run()
    
    def train(self, inputs, labels):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: inputs, self.Y: labels})
     
    def predict(self, inputs):
        return self.sess.run(self.prediction, feed_dict={self.X: inputs})
        

# Training
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    input_batch, target_batch = make_batch(seq_data)
    model = TextLSTM(sess, vocab_size, n_embed, n_step, n_hidden, n_class)
    for epoch in range(1000):
        loss, _ = model.train(input_batch, target_batch)
        if (epoch + 1) % 100 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    predict =  model.predict(input_batch)

inputs = [sen[:3] for sen in seq_data]
for i, input in enumerate(inputs):
    print(input, '->', number_dict[predict[i][0]])

Epoch: 0100 cost = 0.001793
Epoch: 0200 cost = 0.000833
Epoch: 0300 cost = 0.000471
Epoch: 0400 cost = 0.000300
Epoch: 0500 cost = 0.000207
Epoch: 0600 cost = 0.000151
Epoch: 0700 cost = 0.000115
Epoch: 0800 cost = 0.000090
Epoch: 0900 cost = 0.000072
Epoch: 1000 cost = 0.000059
mak -> e
nee -> d
coa -> l
wor -> d
lov -> e
hat -> e
liv -> e
hom -> e
has -> h
sta -> r
